In [1]:
import xarray as xr
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
f = xr.open_dataset('seNorge2018_RR_1957_2017.nc')

In [3]:
f['time'][:]

<xarray.DataArray 'time' (time: 22280)>
array(['1957-01-01T06:00:00.000000000', '1957-01-02T06:00:00.000000000',
       '1957-01-03T06:00:00.000000000', ..., '2017-12-29T06:00:00.000000000',
       '2017-12-30T06:00:00.000000000', '2017-12-31T06:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1957-01-01T06:00:00 ... 2017-12-31T06:00:00
Attributes:
    axis:           T
    long_name:      time
    standard_name:  time
    cell_methods:   time: sum
    bounds:         time_bnds

In [5]:
f_days = f.assign(date = lambda f: f.time.dt.round('D')) 
f_days_of_week = f_days.assign(day_of_week = lambda f: f_days.date.dt.weekday) 

In [6]:
# f_days_of_week

In [1]:
# f.DataArray.dt.weekday

In [10]:
# dimentions:
1195*1550*22280
\

41268130000

In [18]:
f_days_of_week.day_of_week

(22280,)

In [24]:
f_days_of_week['day_of_week']

<xarray.DataArray 'day_of_week' (time: 22280)>
array([1, 2, 3, ..., 4, 5, 6])
Coordinates:
  * time     (time) datetime64[ns] 1957-01-01T06:00:00 ... 2017-12-31T06:00:00

In [ ]:
xmean = f_days_of_week.groupby('day_of_week').mean()